English to Sinhala Translation

Library Imports

In [2]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

Preparation of data

Mounting the google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the data file

In [4]:
text_file = "/content/drive/My Drive/Deep Learning/EnglishToSinhalaDataset.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break

Go.	යන්න.
Hi.	ආයුබෝවන්.
Run.	දුවන්න.
Who?	කව්ද?
Wow!	වාව්!
Fire.	ගින්නක්.
Help.	උදව්.
Hide.	සඟවන්න.
Jump.	පනින්න.
Stay.	රැඳී සිටින්න.
Stop.	නවත්වන්න.
Wait.	ඉන්න.
Begin.	ආරම්භය.
Go on.	දිගටම යන්න.
Hello!	හෙලෝ!
Hurry!	ඉක්මන් කරන්න!
I hid.	මම සැඟවී සිටියෙමි.
I ran.	මම දිව්වා.
I try.	මම උත්සාහ කරමි.
I won.	මම දිනුවා.


In [5]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

No matter how much you try to convince people that chocolate is vanilla, it'll still be chocolate, even though you may manage to convince yourself and a few others that it's vanilla.	චොක්ලට් යනු වැනිලා බව මිනිසුන්ට ඒත්තු ගැන්වීමට ඔබ කොතරම් උත්සාහ කළත්, එය වැනිලා බව ඔබට සහ තවත් කිහිප දෙනෙකුට ඒත්තු ගැන්විය හැකි වුවද, එය තවමත් චොකලට් වනු ඇත.
In 1969, Roger Miller recorded a song called "You Don't Want My Love." Today, this song is better known as "In the Summer Time." It's the first song he wrote and sang that became popular.	1969 දී රොජර් මිලර් "ඔබට මගේ ආදරය අවශ්‍ය නැත" නමින් ගීතයක් පටිගත කළේය. අද මෙම ගීතය වඩාත් ප්‍රචලිත වන්නේ "ගිම්හානයේ" යනුවෙනි. එය ඔහු ලියූ සහ ගායනා කළ පළමු ගීතය ජනප්‍රිය විය.
A child who is a native speaker usually knows many things about his or her language that a non-native speaker who has been studying for years still does not know and perhaps will never know.	ස්වදේශික කථිකයෙකු වන දරුවෙකු සාමාන්‍යයෙන් ඔහුගේ හෝ ඇයගේ භාෂාව පිළිබඳ බොහෝ දේ දන්නා අතර එය වසර ගණනාවක් තිස්ස

Spliting the English and Sinhala translation pairs

In [6]:
text_pairs = []
for line in lines:
    english, sinhala = line.split("\t")
    sinhala = "[start] " + sinhala + " [end]"
    text_pairs.append((english, sinhala))
for i in range(3):
  print(random.choice(text_pairs))

('Do you get along well with your new classmates?', '[start] ඔබ ඔබේ නව පන්තියේ මිතුරන් සමඟ හොඳින් ඇසුරු කරනවාද? [end]')
('I think I lost my keys.', '[start] මම හිතන්නේ මගේ යතුරු නැති වුණා. [end]')
('Chances are that I will be late for work.', '[start] ඔබ බොහෝ විට රැකියාවට ප්රමාද වනු ඇත. [end]')


Randomize the data

In [7]:
import random
random.shuffle(text_pairs)

Splitting the data into training, validation and testing

In [8]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 125603
Training set size: 87923
Validation set size: 18840
Testing set size: 18840


In [9]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

125603

Removing Punctuation

In [10]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [11]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [12]:
f"{3+5}"

'8'

Vectorizing the English and Sinhala Text Pairs

In [13]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

Preparing datasets for the translation task

In [14]:
batch_size = 64
def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "sinhala": spa[:, :-1],
    }, spa[:, 1:])
def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")
inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[50])

inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
({'english': array([[   3,   36,   17, ...,    0,    0,    0],
       [   2, 6699, 5999, ...,    0,    0,    0],
       [  19,  165, 6041, ...,    0,    0,    0],
       ...,
       [  91,   92,    6, ...,    0,    0,    0],
       [   3, 2542,   13, ...,    0,    0,    0],
       [   6,   43,    7, ...,    0,    0,    0]]), 'sinhala': array([[   2,    4,   30, ...,    0,    0,    0],
       [   2, 8114, 6406, ...,    0,    0,    0],
       [   2,    4, 1620, ...,    0,    0,    0],
       ...,
       [   2,   81,   18, ...,    0,    0,    0],
       [   2,    4,   13, ...,    0,    0,    0],
       [   2,   18, 1967, ...,    0,    0,    0]])}, array([[    4,    30, 10102, ...,     0,     0,     0],
       [ 8114,  6406,  5866, ...,     0,     0,     0],
       [    4,  1620,  3296, ...,     0,     0,     0],
       ...,
       [   81,    18,    88, ...,     0,     0,     0],
       [    4,    1

Transformer encoder implemented as a subclassed layer

In [15]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

The Transformer Decoder

In [16]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

Positional Encoding

In [17]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

End To End Transformer

In [18]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

Training the Sequence To Sequence Transformer

In [19]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
1374/1374 [==============================] - 109s 73ms/step - loss: 4.3415 - accuracy: 0.4102 - val_loss: 3.4771 - val_accuracy: 0.4893
Epoch 2/50
1374/1374 [==============================] - 94s 68ms/step - loss: 3.3915 - accuracy: 0.5031 - val_loss: 3.0651 - val_accuracy: 0.5386
Epoch 3/50
1374/1374 [==============================] - 94s 68ms/step - loss: 3.0961 - accuracy: 0.5403 - val_loss: 2.9227 - val_accuracy: 0.5627
Epoch 4/50
1374/1374 [==============================] - 94s 68ms/step - loss: 2.9375 - accuracy: 0.5634 - val_loss: 2.8578 - val_accuracy: 0.5736
Epoch 5/50
1374/1374 [==============================] - 94s 69ms/step - loss: 2.8414 - accuracy: 0.5797 - val_loss: 2.8532 - val_accuracy: 0.5788
Epoch 6/50
1374/1374 [==============================] - 95s 69ms/step - loss: 2.7746 - accuracy: 0.5915 - val_loss: 2.8466 - val_accuracy: 0.5816
Epoch 7/50
1374/1374 [==============================] - 95s 69ms/step - loss: 2.7143 - accuracy: 0.6023 - val_loss: 2.8550 

Testing

In [20]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20
def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Whose lyrics are these?
[start] මේ [UNK] කාගේද [end]
-
It was no big deal, really.
[start] එය ඇත්තටම විශාල නැත [end]
-
Are you ready for the trip?
[start] ඔබ සංචාරය සඳහා සූදානම් කරනවාද [end]
-
I hadn't recognized the importance of this document until you told me about it.
[start] අනතුරට හේතුව මට අද උදෑසන මට [UNK] [end]
-
He fed his dog at the same time every day.
[start] එයා හැමදාම සමහර වෙලාවට මේ දවස්වල මගේ බල්ලා එකම කළා [end]
-
Tom smiled approvingly.
[start] ටොම් කෑ ගැහුවා [end]
-
You broke your leg.
[start] ඔබේ කර ජලය [UNK] [end]
-
Do this work by tomorrow if at all possible.
[start] මේ සියල්ල ක්‍රියා විරහිත [UNK] කළ හැකිද [end]
-
I can't come now.
[start] මට දැන් යන්න බෑ [end]
-
Our marriage is over.
[start] අපේ රැස්වීමට [UNK] [end]
-
What kind of bird is this?
[start] එය කුමන ආකාරයේ [UNK] [end]
-
Why do American parents praise their children?
[start] දරුවන් [UNK] [UNK] [UNK] ඇයි [end]
-
How about stopping the car and taking a rest?
[start] අපි නිදා ගැනීමට යන මෝටර් රථය ගැන මොකද ද